# Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


In [ ]:
!pip install pyconll

In [ ]:
import numpy as np
import pandas as pd
import pyconll
import nltk

from nltk.corpus import brown
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

## загрузка данных

In [ ]:
!mkdir datasets

In [ ]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-09-21 19:11:53--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   182MB/s    in 0.4s    

2021-09-21 19:11:54 (182 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-09-21 19:11:54--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [ ]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')
df_accuracy  = pd.DataFrame(columns=['accuracy'])

In [ ]:
train_data = [[(token.form, token.upos) for token in sent] for sent in full_train]
test_data = [[(token.form, token.upos) for token in sent] for sent in full_test]

test_sent = [[form for form, _ in row] for row in test_data]


In [ ]:
test_sent_choice = np.random.choice(test_sent)
test_sent_choice

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


['Ему', 'обрисовывают', 'человека', ',', 'с', 'ног', 'до', 'головы', '.']

In [ ]:
print(f'train_data: \n{train_data[:2]}\ntest_data:\n{test_data[:2]}')

train_data: 
[[('Анкета', 'NOUN'), ('.', 'PUNCT')], [('Начальник', 'NOUN'), ('областного', 'ADJ'), ('управления', 'NOUN'), ('связи', 'NOUN'), ('Семен', 'PROPN'), ('Еремеевич', 'PROPN'), ('был', 'AUX'), ('человек', 'NOUN'), ('простой', 'ADJ'), (',', 'PUNCT'), ('приходил', 'VERB'), ('на', 'ADP'), ('работу', 'NOUN'), ('всегда', 'ADV'), ('вовремя', 'ADV'), (',', 'PUNCT'), ('здоровался', 'VERB'), ('с', 'ADP'), ('секретаршей', 'NOUN'), ('за', 'ADP'), ('руку', 'NOUN'), ('и', 'CCONJ'), ('иногда', 'ADV'), ('даже', 'PART'), ('писал', 'VERB'), ('в', 'ADP'), ('стенгазету', 'NOUN'), ('заметки', 'NOUN'), ('под', 'ADP'), ('псевдонимом', 'NOUN'), ('"', 'PUNCT'), ('Муха', 'NOUN'), ('"', 'PUNCT'), ('.', 'PUNCT')]]
test_data:
[[('Алгоритм', 'NOUN'), (',', 'PUNCT'), ('от', 'ADP'), ('имени', 'NOUN'), ('учёного', 'NOUN'), ('аль', 'PART'), ('-', 'PUNCT'), ('Хорезми', 'PROPN'), (',', 'PUNCT'), ('-', 'PUNCT'), ('точный', 'ADJ'), ('набор', 'NOUN'), ('инструкций', 'NOUN'), (',', 'PUNCT'), ('описывающих', 'VERB')

## 1.1

In [ ]:
taggers = [UnigramTagger, BigramTagger, TrigramTagger]

for tag_c in taggers:
  tagger = tag_c(train_data)
  print(tag_c.__name__)
  display(tagger.tag(test_sent_choice), tagger.evaluate(test_data))
  df_accuracy.loc[tag_c.__name__, 'accuracy'] = tagger.evaluate(train_data)

UnigramTagger


[('Ему', 'PRON'),
 ('обрисовывают', 'VERB'),
 ('человека', 'NOUN'),
 (',', 'PUNCT'),
 ('с', 'ADP'),
 ('ног', 'NOUN'),
 ('до', 'ADP'),
 ('головы', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

BigramTagger


[('Ему', 'PRON'),
 ('обрисовывают', None),
 ('человека', None),
 (',', 'PUNCT'),
 ('с', 'ADP'),
 ('ног', 'NOUN'),
 ('до', 'ADP'),
 ('головы', 'NOUN'),
 ('.', 'PUNCT')]

0.6963064064974893

TrigramTagger


[('Ему', 'PRON'),
 ('обрисовывают', None),
 ('человека', None),
 (',', None),
 ('с', 'ADP'),
 ('ног', None),
 ('до', None),
 ('головы', None),
 ('.', None)]

0.24808748694099012

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(train_data,  
                     taggers,  
                     backoff = backoff) 

df_accuracy.loc['BackoffTagger', 'accuracy'] = tag.evaluate(test_data)
tag.evaluate(test_data) 

0.9119991237825633

## 1.2

In [ ]:
%%time
train_tok = [form for row in train_data  for form, _ in row]
train_label = ['NO_TAG' if upos is None else upos for row in train_data for _, upos in row]

test_tok = [form for row in test_data for form, _ in row ]
test_label = ['NO_TAG' if upos is None else upos for row in test_data for _, upos in row]

CPU times: user 351 ms, sys: 4.02 ms, total: 355 ms
Wall time: 353 ms


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
def get_accuracy_vec(vectorizer, train_t, test_t, train_l, test_l):
  
  le = LabelEncoder()
  train_enc_l = le.fit_transform(train_l)
  test_enc_l = le.transform(test_l)

  X_train = vectorizer.fit_transform(train_t)
  X_test = vectorizer.transform(test_t)
  print(vectorizer.get_params())
  print(f'{X_train.shape}\n')

  lr = LogisticRegression(random_state=0)
  lr.fit(X_train, train_enc_l)
  pred = lr.predict(X_test)
  vec_name = f"{vectorizer.__name__}:[analyzer:{vectorizer.get_params()['analyzer']},n_features:{vectorizer.get_params()['n_features']}"
  df_accuracy.loc[vec_name, 'accuracy'] = accuracy_score(test_enc_l, pred)
  return f'Accuracy: {accuracy_score(test_enc_l, pred)}'


In [ ]:
vectorizers = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100),
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000),
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000),
               CountVectorizer(ngram_range=(1, 5), analyzer='char'),  #, binary=False, tokenizer=str.split),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),  #, binary=False, tokenizer=str.split),
               ]

In [ ]:
print(get_accuracy_vec(vectorizer=TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),
                         train_t=train_tok,
                         test_t=test_tok,
                         train_l=train_label,
                         test_l=test_label))

{'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}
(871526, 149809)



In [ ]:
for vectorizer in vectorizers:
  print(get_accuracy_vec(vectorizer=vectorizer,
                         train_t=train_tok,
                         test_t=test_tok,
                         train_l=train_label,
                         test_l=test_label))

(871526, 100)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 100, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.686221480807468
(871526, 1000)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 1000, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.8805058470663566
(871526, 1000)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 1000, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.3789303407137802
(871526, 149809)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 5), 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'vocabulary': None}


KeyError: ignored

## 1.3

In [ ]:
df_accuracy

Прирос при равных параметрах у HashingVect при увеличении количеста фичей.
При внесении тех же данных при смене анализатора падение точности.

Небольшой выигрыш CountVectorizer у TfidfVectorizer

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

## 2.1

In [1]:
!pip install corus

     |████████████████████████████████| 83 kB 1.8 MB/s 


In [2]:
import corus

In [3]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-09-22 09:43:11--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   372KB/s    in 5.7s    

2021-09-22 09:43:17 (324 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

In [5]:
!rm collection5.zip

In [6]:
!ls

Collection5  sample_data


In [7]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


Ne5Markup(
    id='last_70',
    text='В.Путин оставил Р.Копина во главе Чукотки\r\n\r\n\r\nПрезидент России Владимир Путин своим указом назначил временно исполняющим обязанности губернатора Чукотского автономного округа Романа Копина. Указ подписан в связи с истечением срока его полномочий на посту губернатора, сообщила пресс-служба Кремля.\r\n\r\nР.Копин назначен временным главой региона до вступления в должность лица, избранного губернатором Чукотки. Президентский указ вступает в силу со дня его подписания.\r\n\r\nР.Копин официально вступил в должность губернатора Чукотки в июле 2008г. после отставки Романа Абрамовича. Его кандидатуру в качестве главы региона предложил тогда еще президент России Дмитрий Медведев.\r\n\r\nДо назначения Р.Копин занимал посты заместителя губернатора и заместителя председателя правительства округа по вопросам промышленной и сельскохозяйственной политики.\r\n',
    spans=[Ne5Span(
         index='T1',
         type='PER',
         start=0,
         stop=7

процедуры обработки взять из вебинарного ноутбука

In [14]:
import requests
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('http://www.labinform.ru/pub/named_entities')

nltk.pos_tag(nltk.word_tokenize(document))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


[('Размеченные', 'JJ'),
 ('коллекции', 'NNP'),
 ('для', 'NNP'),
 ('извлечения', 'NNP'),
 ('именованных', 'NNP'),
 ('сущностей', 'NNP'),
 ('LABINFORM.RU', 'NNP'),
 ('РАЗМЕЧЕННЫЕ', 'NNP'),
 ('КОЛЛЕКЦИИ', 'NNP'),
 ('ДЛЯ', 'NNP'),
 ('ИЗВЛЕЧЕНИЯ', 'NNP'),
 ('ИМЕНОВАННЫХ', 'NNP'),
 ('СУЩНОСТЕЙ', 'NNP'),
 ('Коллекция', 'NNP'),
 ('Named_Entities_5', 'NNP'),
 ('Размеченная', 'NNP'),
 ('коллекция', 'NNP'),
 ('Named_Entities_5', 'NNP'),
 ('создана', 'NNP'),
 ('для', 'NNP'),
 ('оценки', 'NNP'),
 ('качества', 'NNP'),
 ('алгоритмов', 'NNP'),
 ('автоматического', 'NNP'),
 ('извлечения', 'NNP'),
 ('именованных', 'NNP'),
 ('сущностей', 'NNP'),
 ('из', 'NNP'),
 ('текстов', 'NNP'),
 ('на', 'NNP'),
 ('русском', 'NNP'),
 ('языке', 'NNP'),
 ('.', '.'),
 ('В', 'VB'),
 ('качестве', 'JJ'),
 ('основы', 'NNP'),
 ('для', 'NNP'),
 ('разметки', 'NNP'),
 ('этих', 'NNP'),
 ('коллекций', 'NNP'),
 ('взята', 'NNP'),
 ('коллекция', 'NNP'),
 ('Persons-1000', 'NNP'),
 (',', ','),
 ('подготовленная', 'NNP'),
 ('Исследовател

In [16]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


{('DOI', 'ORGANIZATION'),
 ('Geopolit', 'ORGANIZATION'),
 ('ISMW', 'ORGANIZATION'),
 ('Intelligence', 'GPE'),
 ('Loc', 'ORGANIZATION'),
 ('Loc Имена', 'PERSON'),
 ('Loukachevitch N.', 'PERSON'),
 ('MEDIA', 'ORGANIZATION'),
 ('Org', 'ORGANIZATION'),
 ('Org Москвы', 'PERSON'),
 ('Org Правительства', 'PERSON'),
 ('Org Разметка', 'PERSON'),
 ('PDF', 'ORGANIZATION'),
 ('Russian', 'GPE'),
 ('Social Media', 'ORGANIZATION'),
 ('Библиотека', 'ORGANIZATION'),
 ('Вложенные', 'PERSON'),
 ('Коллекция', 'PERSON'),
 ('Ленина', 'PERSON'),
 ('Лукашевич', 'PERSON'),
 ('Можарова В.А.', 'PERSON'),
 ('Москва', 'PERSON'),
 ('Типы', 'PERSON'),
 ('сущностейКоличество Pers10623', 'ORGANIZATION')}

In [18]:
!pip install razdel

In [19]:
from razdel import tokenize
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [20]:
import pandas as pd
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [21]:
df_words['tag'].value_counts()

OUT         219121
PER          21177
ORG          13650
LOC           4562
GEOPOLIT      4354
MEDIA         2482
Name: tag, dtype: int64

In [22]:
df_words.head(3)

,word,tag
0,Египетская,OUT
1,армия,OUT
2,выдвинула,OUT


## 2.2

In [ ]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
#!python -m deeppavlov install squad_bert
#!python -m deeppavlov install ner_ontonotes

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

## 2.3

In [23]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model



In [24]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [25]:
train_x.apply(len).max(axis=0)

55

In [26]:
valid_x

83853            власти
41607           третьей
89875                 <
135481                в
153019                "
              ...      
156005    собственность
77633          отставку
240567         сегмента
184701         Сердюков
142202         Госстроя
Name: word, Length: 66337, dtype: object

In [27]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [37]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [39]:
len(vectorize_layer.get_vocabulary())

29913

In [40]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [41]:
mmodel = modelNER()

In [42]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [43]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12439/12439 [==============================] - 246s 20ms/step - loss: 0.2979 - accuracy: 0.9130 - val_loss: 0.2077 - val_accuracy: 0.9398
Epoch 2/3
12439/12439 [==============================] - 240s 19ms/step - loss: 0.1281 - accuracy: 0.9616 - val_loss: 0.2154 - val_accuracy: 0.9429
Epoch 3/3
12439/12439 [==============================] - 237s 19ms/step - loss: 0.1108 - accuracy: 0.9648 - val_loss: 0.2674 - val_accuracy: 0.8885


In [44]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=1,
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [45]:
mmodel = modelNER()

In [46]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [47]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12439/12439 [==============================] - 243s 19ms/step - loss: 0.2965 - accuracy: 0.9132 - val_loss: 0.2077 - val_accuracy: 0.9405
Epoch 2/3
12439/12439 [==============================] - 237s 19ms/step - loss: 0.1268 - accuracy: 0.9620 - val_loss: 0.3428 - val_accuracy: 0.8957
Epoch 3/3
12439/12439 [==============================] - 238s 19ms/step - loss: 0.1108 - accuracy: 0.9651 - val_loss: 0.2365 - val_accuracy: 0.8966
